In [1]:
import pandas as pd
import numpy as np
import  seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as st  
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
#leemos los datos (data frame del costo del personal medico)
datos=pd.read_csv('costo_personal.csv')

In [3]:
#visualizamos los 5 primeros valors del dataframe
datos.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
#evaluamos si hay valores nan en el dataframe
check_for_nan = datos.isnull().values.any()
print (check_for_nan)

False


In [5]:
#vemos q tipo de informacion las variables del data frame
datos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
#creamos nustra primera versión del modelo con todas las variables , utilizando la libreria statsmodels
#ya q nos brinda simplicidad para tratar las variables dummy y una mayor información en cuanto metricas,
#comparando con sklearn

In [6]:
modelo=smf.ols(formula= 'charges ~  C( smoker + region ) + age + bmi + children ',data=datos )
modelo=modelo.fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     414.3
Date:                Mon, 30 May 2022   Prob (F-statistic):               0.00
Time:                        19:36:14   Log-Likelihood:                -13530.
No. Observations:                1338   AIC:                         2.708e+04
Df Residuals:                    1327   BIC:                         2.714e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                          -1.137e+04    981.483    -11.584      0.000   -1.33e+04   -9444.416
C(smoker + region)[T.nonorthwest]   -510.0169    523.317     -0.975      0.330   -1536.635     516.601
C(smoker + region)[T.nosoutheast]  -2306.3537    533.502     -4.323      0.000   -3352.954   -1259.753
C(smoker + region)[T.nosouthwest]  -1747.6134    524.342     -3.333      0.001   -2776.244    -718.983
C(smoker + region)[T.yesnortheast]   2.11e+04    821.651     25.680      0.000    1.95e+04    2.27e+04
C(smoker + region)[T.yesnorthwest]  2.103e+04    870.931     24.150      0.000    1.93e+04    2.27e+04
C(smoker + region)[T.yessoutheast]  2.438e+04    738.029     33.037      0.000    2.29e+04    2.58e+04
C(smoker + region)[T.yessouthwest]  2.334e+04    873.118     26.734      0.000    2.16e+04    2.51e+04
age                                  256.4991     11.768     21.796      0.000     233.413     279.585
bmi                                  336.9566     28.232     11.935      0.000     281.573     392.340
children                             487.7646    136.137      3.583      0.000     220.698     754.832
==============================================================================
Omnibus:                      324.302   Durbin-Watson:                   2.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              844.303
Skew:                           1.269   Prob(JB):                    4.59e-184
Kurtosis:                       5.951   Cond. No.                         390.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#se rechaza la hipótesis nula ya que el valor de Prob (F-statistic) es casi igual a 0, esto nos indica que existe alguna
#variable explicativa en el modelo.Posteriomente analizamos que variables son significativas(observamos el p value), 
#tomando como referencia 0,05 , cualquier valor menor lo descartamos.

In [ ]:
#comenzamos ah quitar variables y ver como varia nuestro R2.
#la primer variables que quitamos para analizar su importancia es region, usando el la idea que no debería influenciar 
#mucho en el costo, ya q hablamos del mismo país.

In [7]:
modelo=smf.ols(formula= 'charges ~  C( smoker  ) + age + bmi + children', data=datos )
modelo=modelo.fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     998.1
Date:                Mon, 30 May 2022   Prob (F-statistic):               0.00
Time:                        19:36:21   Log-Likelihood:                -13551.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1333   BIC:                         2.714e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         -1.21e+04    941.984    -12.848      0.000    -1.4e+04   -1.03e+04
C(smoker)[T.yes]  2.381e+04    411.220     57.904      0.000     2.3e+04    2.46e+04
age                257.8495     11.896     21.675      0.000     234.512     281.187
bmi                321.8514     27.378     11.756      0.000     268.143     375.559
children           473.5023    137.792      3.436      0.001     203.190     743.814
==============================================================================
Omnibus:                      301.480   Durbin-Watson:                   2.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              722.157
Skew:                           1.215   Prob(JB):                    1.53e-157
Kurtosis:                       5.654   Cond. No.                         292.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
modelo=smf.ols(formula= 'charges ~  C( smoker  ) + age + bmi ' , data=datos )
modelo=modelo.fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     1316.
Date:                Mon, 30 May 2022   Prob (F-statistic):               0.00
Time:                        19:36:27   Log-Likelihood:                -13557.
No. Observations:                1338   AIC:                         2.712e+04
Df Residuals:                    1334   BIC:                         2.714e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept        -1.168e+04    937.569    -12.454      0.000   -1.35e+04   -9837.561
C(smoker)[T.yes]  2.382e+04    412.867     57.703      0.000     2.3e+04    2.46e+04
age                259.5475     11.934     21.748      0.000     236.136     282.959
bmi                322.6151     27.487     11.737      0.000     268.692     376.538
==============================================================================
Omnibus:                      299.709   Durbin-Watson:                   2.077
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              710.137
Skew:                           1.213   Prob(JB):                    6.25e-155
Kurtosis:                       5.618   Cond. No.                         289.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#notamos q al quitar la variable children disminuye muy poco el R2, pero se la decidio conserar ya que se penso que una persona
# que tenga hijos tendría que influir en costo medico de esa persona .

In [ ]:
#ahora solo para comparar quitamos una variables que consideramos 
#que es importante,hablamos de la variable smoker (volvemos a incluier region en el modelo luego de la comprabación) 

In [9]:
modelo=smf.ols(formula= 'charges ~  C( region) + age + bmi + children ',data=datos )
modelo=modelo.fit()
modelo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     31.26
Date:                Mon, 30 May 2022   Prob (F-statistic):           2.77e-35
Time:                        19:36:34   Log-Likelihood:                -14389.
No. Observations:                1338   AIC:                         2.879e+04
Df Residuals:                    1331   BIC:                         2.883e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept              -5978.2665   1824.861     -3.276      0.001   -9558.184   -2398.349
C(region)[T.northwest] -1037.2971    892.474     -1.162      0.245   -2788.105     713.511
C(region)[T.southeast]    69.0309    896.569      0.077      0.939   -1689.811    1827.873
C(region)[T.southwest] -1613.4288    895.607     -1.801      0.072   -3370.385     143.527
age                      241.0014     22.291     10.812      0.000     197.272     284.730
bmi                      319.7839     53.546      5.972      0.000     214.740     424.828
children                 565.4871    258.253      2.190      0.029      58.861    1072.114
==============================================================================
Omnibus:                      326.326   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              606.788
Skew:                           1.521   Prob(JB):                    1.73e-132
Kurtosis:                       4.279   Cond. No.                         307.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# efectivamente es una variable muy importante en el modelo.
# ahora crearemos un modelo con la libreria sklearn con las variables que consideramos mas relevantes
#pero antes tenemos que convertir la variable smoker a variable numerica.


In [10]:
datos = pd.get_dummies(datos, columns = ["smoker"], drop_first = True)
datos.head()


,age,sex,bmi,children,region,charges,smoker_yes
0,19,female,27.900,0,southwest,16884.92400,1
1,18,male,33.770,1,southeast,1725.55230,0
2,28,male,33.000,3,southeast,4449.46200,0
3,33,male,22.705,0,northwest,21984.47061,0
4,32,male,28.880,0,northwest,3866.85520,0


In [ ]:
#dividimos los datos en datos de entrenamientos y datos de prueba, tomando un 90% de los datos para entrenar el modelo

In [11]:
X=datos[['age','bmi','children','smoker_yes']]
y= datos['charges']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

#creamos y entrenamos nuestro modelo
reg=LinearRegression()
reg=reg.fit(X_train,y_train)

In [12]:
print(reg.coef_)
print(reg.intercept_)

[  255.02028276   307.12157843   455.63164974 23479.47127718]
-11510.089698361979


In [13]:
#utilizamos validación cruzada pare evaluar nuestro modelo.
print(cross_val_score(reg,datos[['age','bmi','children','smoker_yes']].values,datos["charges"].values,
cv=10).mean())

0.744650573783781


In [14]:
#hacemos predicciones con los datos que reservamos como datos de prueba.
y_pred = reg.predict(X_test)
y_pred

array([11481.66832366,  9951.65599975, 37580.00527998, 15833.47373036,
        7039.48921321,  3725.32365454,  1162.1797278 , 14037.45830298,
        8825.71888026,  7303.16585048,  4362.80485498, 10204.91552572,
        9094.34930527,  4371.70520448, 27607.89419871, 11256.36303762,
       11260.32405483,  5554.44938646,  8078.32173101, 26493.81286825,
       33501.75883639, 14069.16121837, 11233.34373936, 32502.5283964 ,
        4471.85222606,  8572.57513053,   940.61031007, 10192.62864961,
        4300.84849611, 10242.7588597 ,  8868.93464656, 40106.19164822,
       15787.51486625, 13971.66072908, 24392.00315453,  4718.42447011,
       12854.93725693, 30744.80660083, 33147.3659219 ,  3817.46451868,
        3768.53942085,  4466.47540314, 30026.25349293, 39021.49920946,
       27877.17683376,  4793.33272229, 11155.01291673,  7738.29816383,
        3335.93347294, 10846.47351962,  5063.17063721,  3280.09630895,
       33265.39099724, 37721.16542982, 16152.55205394,  6513.33135072,
      

In [15]:
#conclusión
# el modelo que se obtuvo es capaz de predecir el 74% de las predicciones, y esta dado por la función:
#  y= 255.020283age + 307.121578bmi + 455.63164974children + 23479.471277smoker_yes